# IROS 2023 - Hands-on Robot Control in CRAM

This tutorial will lead you through the IROS 2023 Tutorial on Robot Programming with the Cognitive Robot Abstract Machine (CRAM). We'll start off with the setup, then list the lectures to follow.


## Setup

This tutorial shows the use of the plan executive PyCRAM. The whole tutorial will take place in this very JupyterHub.  Open a new Launcher by opening a new tab, or with Ctrl-Shift-L. From the **Launcher** you can open a new bash terminal, the RvizWeb visualization or create a new Notebook to execute Python code from.

Enter the Launcher and **start RvizWeb**. You can shove it over for a split screen with the Notebook, and close the navigation bar on the very left for a better view. RvizWeb should only show an empty grid right now.

## Lectures

1. [Bullet World](bullet_world.ipynb) is our physics environment. Get familiar with spawning objects and poses.
2. [Poses](pose.ipynb) will refine your understanding of applied 3D linear algebra.
3. [Object Designators](object_designator.ipynb) are the vague description of objects in the world.
4. [Motion Designators](motion_designator.ipynb) are the lowest-level instructions for the robot.
5. [Location Designators](location_designator.ipynb) are underspecified regions that span up a distribution of poses, also known as a Costmap.
6. [Action Designators](action_designator.ipynb) are descriptions of actions. Resolve and perform them to make the robot do things.
7. [Task Trees](minimal_task_tree.ipynb) are one representation of the execution trace. It contains every action executed and the parameters used within.
8. [Object Relational Mapping](orm_example.ipynb) is one of our mechanisms, to store a robots experience during its execution.
9. [Custom Resolver](custom_resolver.ipynb) shows a location resolver that reduces the sample space through prior experience.


## Troubleshoot

When you start a new Notebook, refresh RvizWeb with the circled arrow button in the top right. If you have any leftovers from a Notebook, restart the launchfile and kill all kernels via the `Kernel` menu element in the top bar. Restart the current kernel if the visualization gets stuck as well.

If you are stuck with weird changes in your Notebook, you can revert it to its original state with right-click -> Git -> Discard.


## Latest developments

Following you have an example of solution iterators. As you have surely recognized, finding suitable locations is computationally heavy. We want to solve this by finding the next solution iteratively. Costmaps can be designed as iterators, to retreive new solutions if the current one is bad.

First we initialize the world, robot and environment.

In [ ]:
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.pose import Pose
from pycram.bullet_world import BulletWorld, Object
from pycram.process_module import simulated_robot

world = BulletWorld("DIRECT")
robot = Object("pr2", "robot", "pr2.urdf", pose=Pose([1, 2, 0]))
apartment = Object("apartment", "environment", "apartment.urdf")

robot_desig = BelieveObject(names=["pr2"])
apartment_desig = BelieveObject(names=["apartment"])

Retreive the handle of a drawer and create a generator for locations to access it.

In [ ]:
with simulated_robot:
    handle_desig = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig.resolve())
    drawer_open_loc_solutions = \
        iter(AccessingLocation(handle_desig=handle_desig.resolve(),
                               robot_desig=robot_desig.resolve()))

Then iterate through the solutions.

In [ ]:
with simulated_robot:
    drawer_open_loc = next(drawer_open_loc_solutions)

This is the underlying technique used for finding solutions for the OpenAction, as described in the lecture on action designators.